# EDA and Feature engineering


## Intro and Goals

### Analysing the booking completetion of booking tickets
#### Problem statement and Scope
Airline and travel booking platforms often experience a significant number of incomplete bookings — customers start the booking process (selecting routes, seats, and preferences) but abandon before final payment.
This project aims to predict whether a booking will be completed using historical booking data containing features like purchase lead time, travel routes, flight durations, and seat preferences.

By identifying factors that influence booking completion, the business can:

Optimize the booking process to reduce drop-offs.

Target at-risk customers with personalized reminders or offers.

Improve marketing ROI by focusing on high-likelihood customers.

Goal: Build a machine learning model that predicts booking completion with high accuracy, while ensuring interpretability for business stakeholders.

Success Criteria:

Primary: Achieve at least 85% accuracy or 0.85 F1-score on test data.

Secondary: Identify the top 5 most influential factors affecting booking completion.


## Load and data checks

In [18]:
#imports

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np


In [19]:
df=pd.read_csv("customer_booking.csv",encoding="ISO-8859-1")
df=df.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   num_passengers         50000 non-null  int64  
 1   sales_channel          50000 non-null  object 
 2   trip_type              50000 non-null  object 
 3   purchase_lead          50000 non-null  int64  
 4   length_of_stay         50000 non-null  int64  
 5   flight_hour            50000 non-null  int64  
 6   flight_day             50000 non-null  object 
 7   route                  50000 non-null  object 
 8   booking_origin         50000 non-null  object 
 9   wants_extra_baggage    50000 non-null  int64  
 10  wants_preferred_seat   50000 non-null  int64  
 11  wants_in_flight_meals  50000 non-null  int64  
 12  flight_duration        50000 non-null  float64
 13  booking_complete       50000 non-null  int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 5.3+ 

In [20]:
df.describe()

,num_passengers,purchase_lead,length_of_stay,flight_hour,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete
count,50000.000000,50000.000000,50000.00000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,1.591240,84.940480,23.04456,9.06634,0.668780,0.296960,0.427140,7.277561,0.149560
std,1.020165,90.451378,33.88767,5.41266,0.470657,0.456923,0.494668,1.496863,0.356643
min,1.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,4.670000,0.000000
25%,1.000000,21.000000,5.00000,5.00000,0.000000,0.000000,0.000000,5.620000,0.000000
50%,1.000000,51.000000,17.00000,9.00000,1.000000,0.000000,0.000000,7.570000,0.000000
75%,2.000000,115.000000,28.00000,13.00000,1.000000,1.000000,1.000000,8.830000,0.000000
max,9.000000,867.000000,778.00000,23.00000,1.000000,1.000000,1.000000,9.500000,1.000000


## Target Balance

In [21]:
import pandas as pd


if df['booking_complete'].dtype != 'int64' and df['booking_complete'].dtype != 'float64':
    
    df['booking_complete'] = df['booking_complete'].map({True:1, False:0, 'Yes':1, 'No':0, 'Y':1, 'N':0}).fillna(df['booking_complete'])
df['booking_complete'] = df['booking_complete'].astype(int)

# Class balance
counts = df['booking_complete'].value_counts(dropna=False)
rates = df['booking_complete'].value_counts(normalize=True, dropna=False).mul(100).round(2)

print("Booking completion class counts:")
print(counts.to_string())
print("\nBooking completion class rates (%):")
print(rates.to_string())

# Small, neat display as a DataFrame
class_balance = pd.DataFrame({
    'count': counts,
    'rate_%': rates
}).sort_index()
display(class_balance)


Booking completion class counts:
booking_complete
0    42522
1     7478

Booking completion class rates (%):
booking_complete
0    85.04
1    14.96


,count,rate_%
booking_complete,,
0,42522,85.04
1,7478,14.96


## Type cleaning

In [ ]:
# map the days with their corresponding order in a week 
df['flight_day'].value_counts()
mapping={'Mon':1,'Tue':2,'Wed':3,'Thu':4,'Fri':5,'Sat':6,'Sun':7}
df['flight_day_numerical']=df['flight_day'].map(mapping)
df['flight_day_numerical']

0        6
1        6
2        3
3        6
4        3
        ..
49995    6
49996    7
49997    6
49998    1
49999    4
Name: flight_day_numerical, Length: 50000, dtype: int64

In [28]:
df['flight_duration']=df['flight_duration'].clip(lower=0)
df['flight_hour']=df['flight_hour'].clip(lower=0)
df['num_passengers']=df['num_passengers'].clip(lower=1)
df['length_of_stay']=df['length_of_stay'].clip(lower=0)
df['purchase_lead']=df['purchase_lead'].clip(lower=0)

In [29]:
num_cols = df.select_dtypes(include=['number']).columns.tolist()
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()

df[num_cols] = df[num_cols].apply(lambda s: s.fillna(s.median()))
df[cat_cols] = df[cat_cols].apply(lambda s: s.fillna('Unknown'))

In [30]:
display(df.isna().sum().sort_values(ascending=False).head(15))

num_passengers           0
sales_channel            0
trip_type                0
purchase_lead            0
length_of_stay           0
flight_hour              0
flight_day               0
route                    0
booking_origin           0
wants_extra_baggage      0
wants_preferred_seat     0
wants_in_flight_meals    0
flight_duration          0
booking_complete         0
flight_day_numerical     0
dtype: int64

## Bins and dayparts

In [37]:


# Lead bins
lead_bins=[-np.inf,3,7,14,30,90,np.inf]
lead_bins_label=['0-3','4-7','8-14','15-30','31-90','90+']
df['lead_bins']=pd.cut(df['purchase_lead'].clip(lower=0),bins=lead_bins,labels=lead_bins_label,include_lowest=True)
# length of stay bins
stay_bins=[-np.inf,3,7,14,30,90,np.inf]
stay_bins_label=['0-3','4-7','8-14','15-30','31-90','90+']
df['stay_bin']=pd.cut(df['length_of_stay'].clip(lower=0),bins=stay_bins,labels=stay_bins_label,include_lowest=True)
#part of the day
def day_part(h):
    if 0 <= h <= 5: return 'night'
    if 6 <= h <= 11: return 'morning'
    if 12 <= h <= 17: return 'afternoon'
    return 'evening'
df['daypart'] = df['flight_hour'].apply(day_part)





## Completion rate plots

## Numeric vs Target plot

## Top 5  EDA insights and  Hypothese